<a href="https://colab.research.google.com/github/MattiaPOLI/DR14/blob/nico/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# ~-----enable Colab to load CSV-----
# import useful libraries
from google.colab import files

# specify the file to load
uploaded = files.upload()

Saving Sky.csv to Sky.csv


In [0]:
# ~-----load desired data-----
# import useful libraries
import pandas as pd
import io

# store data in a Pandas dataframe
df = pd.read_csv(io.StringIO(uploaded['Sky.csv'].decode('utf-8')))
df.head

<bound method NDFrame.head of              objid          ra        dec         u         g         r  \
0     1.237650e+18  183.531326   0.089693  19.47406  17.04240  15.94699   
1     1.237650e+18  183.598371   0.135285  18.66280  17.21449  16.67637   
2     1.237650e+18  183.680207   0.126185  19.38298  18.19169  17.47428   
3     1.237650e+18  183.870529   0.049911  17.76536  16.60272  16.16116   
4     1.237650e+18  183.883288   0.102557  17.55025  16.26342  16.43869   
5     1.237650e+18  183.847174   0.173694  19.43133  18.46779  18.16451   
6     1.237650e+18  183.864379   0.019201  19.38322  17.88995  17.10537   
7     1.237650e+18  183.900081   0.187473  18.97993  17.84496  17.38022   
8     1.237650e+18  183.924588   0.097246  17.90616  16.97172  16.67541   
9     1.237650e+18  183.973498   0.081626  18.67249  17.71375  17.49362   
10    1.237650e+18  183.979195   0.135998  19.29772  17.80227  17.18266   
11    1.237650e+18  184.085331   0.112110  18.83307  17.51785  16.9427